# Data Preparation

In [1]:
#imports
import pandas as pd
import icd10
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
predict = 'F60'
exact_match = False
diag_threshold = 3 # number of diagnoses for a patient to be included in reporting, ignored if < 0
from_year = 2019
to_year = 2020
generate_more_data = False
balance_dataset = True
predict_name = icd10.find(predict).description

In [3]:
q1_df = pd.read_csv("original data/q1.csv", dtype = {'Patient Number': int, 'Billing Year': int, 'Billing Month': int, 'Diagnosis': 'string', 'Birth Year': int, 'Sex': 'string'}, sep=";", error_bad_lines=False, warn_bad_lines=True)
q2_df = pd.read_csv("original data/q2.csv", dtype = {'Patient Number': int, 'Billing Year': int, 'Billing Month': int, 'Diagnosis': 'string', 'Birth Year': 'string', 'Sex': 'string'}, sep=";", error_bad_lines=False, warn_bad_lines=True)
q3_df = pd.read_csv("original data/q3.csv", dtype = {'Patient Number': int, 'Billing Year': int, 'Billing Month': int, 'Diagnosis': 'string', 'Birth Year': int, 'Sex': 'string'}, sep=";", error_bad_lines=False, warn_bad_lines=True)

In [4]:
q1_df = q1_df[~q1_df["Diagnosis"].str.contains("#", na=False)]
q2_df = q2_df[~q2_df['Birth Year'].str.contains("#NV", na=False)]

In [5]:
df = q1_df.append(q2_df).append(q3_df)

In [6]:
#df = df[(df['Billing Year'] >= from_year) & (df['Billing Year'] < to_year)]
df.groupby('Patient Number').count()

,Billing Year,Billing Month,Diagnosis,Birth Year,Sex
Patient Number,,,,,
10000005,57,57,57,57,57
10000006,42,42,42,42,42
10000008,152,152,152,152,152
10000010,24,24,24,24,24
10000011,36,36,36,36,36
...,...,...,...,...,...
12069855,3,3,3,3,3
12070228,3,3,3,3,3
12073327,9,9,9,9,9


In [7]:
if not exact_match:
    df['Diagnosis'] = df['Diagnosis'].str.replace(predict+'\.[0-9]+', predict, regex=True)

#################################################################################
#                                 filter year                                   #
#################################################################################
if generate_more_data:
    df_temp = df[(df['Billing Year'] >= from_year) & (df['Billing Year'] < to_year)]
    df_predict = df[~df['Patient Number'].isin(df_temp['Patient Number'])]
    df_predict = df_predict[df_predict['Diagnosis'].str.contains(predict)]
    df = df_temp.append(df_predict)
else:
    df = df[(df['Billing Year'] >= from_year) & (df['Billing Year'] < to_year)]

In [8]:
df

,Patient Number,Billing Year,Billing Month,Diagnosis,Birth Year,Sex
47,10000784,2019,1,S92.1,1999,F
48,10000784,2019,2,Z01.8,1999,F
49,10000784,2019,2,Z47.0,1999,F
50,10000784,2019,3,S92.1,1999,F
51,10000784,2019,3,Z09.4,1999,F
...,...,...,...,...,...,...
1143958,11674206,2019,10,F50.2,2002,F
1143962,11674226,2019,10,F50.0,2002,F
1143966,11674591,2019,10,M43.06,2002,F
1143994,11675197,2019,10,F50.0,2002,F


In [9]:
if generate_more_data:
    abc = df[~df['Patient Number'].isin(df_temp['Patient Number'])]
    abc[abc['Diagnosis'].str.contains(predict)].groupby('Patient Number').size()
#else:
    #df[['Diagnosis'].str.contains(predict)].groupby('Patient Number').size()

In [10]:
df_crosstab = pd.crosstab([df['Patient Number'], df['Birth Year'], df['Sex']], df['Diagnosis'], margins=True, margins_name='Total')#normalize='index')

In [11]:
if diag_threshold > 0:
    df_crosstab = df_crosstab[df_crosstab['Total'] > diag_threshold]

In [12]:
df_crosstab

,,Diagnosis,A00,A02.0,A02.1,A02.8,A03.0,A03.1,A03.9,A04,A04.0,A04.5,...,Z97.0,Z97.1,Z97.4,Z97.5,Z97.8,Z98,Z98.1,Z98.8,Z99.9,Total
Patient Number,Birth Year,Sex,,,,,,,,,,,,,,,,,,,,,
10000005,1939,F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,21
10000006,1928,F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
10000008,1923,F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,27
10000011,1939,F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
10000023,1932,F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12037800,1988,F,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16
12037993,1989,M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
12066834,1959,M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8


In [13]:
df_crosstab = df_crosstab.drop(labels='Total', axis=0)
df_crosstab = df_crosstab.drop(labels='Total', axis=1)
df_crosstab = df_crosstab.reset_index()

In [14]:
df_crosstab

Diagnosis,Patient Number,Birth Year,Sex,A00,A02.0,A02.1,A02.8,A03.0,A03.1,A03.9,...,Z96.9,Z97.0,Z97.1,Z97.4,Z97.5,Z97.8,Z98,Z98.1,Z98.8,Z99.9
0,10000005,1939,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10000006,1928,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10000008,1923,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000011,1939,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000023,1932,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62887,12033371,1991,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62888,12037800,1988,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62889,12037993,1989,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62890,12066834,1959,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
if balance_dataset:
    df_crosstab_predict = []
    for i in df_crosstab.columns[df_crosstab.columns.str.contains(predict)]:
        df_crosstab_predict.append(df_crosstab[df_crosstab[i] > 0])
    
    df_crosstab_predict = pd.concat(df_crosstab_predict, ignore_index=True)
    df_crosstab_control = df_crosstab[~df_crosstab['Patient Number'].isin(df_crosstab_predict['Patient Number'])]
    df_pnr_predict = df_crosstab_predict['Patient Number']
    df_pnr_control = df_crosstab_control['Patient Number']
    
    predict_df = df[df['Patient Number'].isin(df_pnr_predict)]
    control_df = df[df['Patient Number'].isin(df_pnr_control)]
    control_df = control_df.sample(n = len(predict_df.groupby('Patient Number').size()))
    df_crosstab_control = df_crosstab_control.sample(n = len(df_crosstab_predict))
    df_crosstab = df_crosstab_predict.append(df_crosstab_control)


In [16]:
df_crosstab = df_crosstab.loc[:, (df_crosstab != 0).any(axis=0)]

In [17]:
df_crosstab[predict] = df_crosstab[predict].apply(lambda x: x if x == 0 else 1) # if predict diag is present, set it to 1

In [18]:
df_crosstab.to_csv('transformed data/ml_dataset_'+predict+'_crosstab.csv', sep=';', index=False)

In [19]:
pd.read_csv('transformed data/ml_dataset_'+predict+'_crosstab.csv', sep=';', index_col=0)

,Birth Year,Sex,A04.8,A08,A08.3,A08.4,A09,A41.9,A49,A56,...,Z76.0,Z80.3,Z81.3,Z85.0,Z86.1,Z93.3,Z95.1,Z95.5,Z96.1,Z96.6
Patient Number,,,,,,,,,,,,,,,,,,,,,
10001626,1999,F,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10001638,1999,M,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10001662,1999,M,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10002471,1994,F,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10003411,1999,M,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10006367,1988,M,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10614262,1970,F,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11114011,1972,F,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_crosstab_predict

Diagnosis,Patient Number,Birth Year,Sex,A00,A02.0,A02.1,A02.8,A03.0,A03.1,A03.9,...,Z96.9,Z97.0,Z97.1,Z97.4,Z97.5,Z97.8,Z98,Z98.1,Z98.8,Z99.9
0,10001626,1999,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10001638,1999,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10001662,1999,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10002471,1994,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10003411,1999,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,11658218,1999,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296,11665348,1996,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
297,11669170,1979,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,11791976,1997,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_crosstab_control

Diagnosis,Patient Number,Birth Year,Sex,A00,A02.0,A02.1,A02.8,A03.0,A03.1,A03.9,...,Z96.9,Z97.0,Z97.1,Z97.4,Z97.5,Z97.8,Z98,Z98.1,Z98.8,Z99.9
19677,10101751,1971,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26915,10153165,1975,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13134,10055148,1952,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4734,10027170,1958,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26152,10149007,1969,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,10006367,1988,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45444,10614262,1970,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53162,11114011,1972,F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51279,10975722,1994,M,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
